# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Output File (CSV)
weather_df=pd.read_csv("../output_data/cities.csv")
weather_df.head()
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             684 non-null    object 
 1   Country          680 non-null    object 
 2   Lat              684 non-null    float64
 3   Lng              684 non-null    float64
 4   Max Temp (F)     684 non-null    float64
 5   Humidity (%)     684 non-null    int64  
 6   Cloudiness (%)   684 non-null    int64  
 7   Wind Speed(mph)  684 non-null    float64
 8   Date             684 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 48.2+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Store humidity and 'Lat' and 'Lng' into  locations
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity (%)"].astype(float)

In [10]:
# Create Heatmap layer for humidity
figure_layout = {
    "width": "1000px",
    "height": "650px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto",
}

fig = gmaps.figure(layout=figure_layout, center=(20,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=int(humidity.max()),
                                 point_radius = 2)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='650px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
ideal_min_temp = 73
ideal_max_temp = 83
ideal_max_wind = 5
ideal_max_humidity = 70
ideal_max_cloudiness = 25
hotel_df = weather_df[(weather_df["Max Temp (F)"]>=ideal_min_temp) & (weather_df["Max Temp (F)"]<=ideal_max_temp) & 
                       (weather_df["Wind Speed(mph)"]<ideal_max_wind) & (weather_df["Cloudiness (%)"]<ideal_max_cloudiness) & 
                       (weather_df["Humidity (%)"]<ideal_max_humidity)].reset_index(drop=True)
hotel_df

,City,Country,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed(mph),Date
0,santa barbara,US,34.73,-120.03,73.00,53,1,4.70,1603490037
1,salalah,OM,17.02,54.09,75.20,44,0,2.24,1603489665
2,san jose,US,37.34,-121.89,75.00,43,20,4.70,1603489777
3,laguna,US,38.42,-121.42,79.00,15,1,4.70,1603489646
4,noumea,NC,-22.28,166.46,82.40,54,0,4.70,1603489510
5,kalabo,ZM,-14.97,22.68,73.74,26,4,3.67,1603490151
6,hilton head island,US,32.22,-80.75,78.80,69,20,4.70,1603489934
7,uvira,CD,-3.41,29.15,74.52,60,6,3.74,1603490210
8,balaka,MW,-14.98,34.96,73.87,47,0,0.65,1603490225


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df['Hotel Nearby'] = ""

In [23]:
# find the nearby hotel of each city based on coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():

    # get restaurant type from df
    location = f"{row['Lat']},{row['Lng']}"

    # add keyword to params dict
    params["location"] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"First hotel for the {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
                
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: santa barbara.
First hotel for the santa barbara is Upper Ballard Campground.
------------
Retrieving Results for Index 1: salalah.
First hotel for the salalah is HAMDAN PLAZA HOTEL.
------------
Retrieving Results for Index 2: san jose.
First hotel for the san jose is The Westin San Jose.
------------
Retrieving Results for Index 3: laguna.
First hotel for the laguna is Holiday Inn Express & Suites Elk Grove Central - Hwy 99.
------------
Retrieving Results for Index 4: noumea.
First hotel for the noumea is Hôtel Le Lagon.
------------
Retrieving Results for Index 5: kalabo.
First hotel for the kalabo is Sikakubete Guest House Kalabo Town Council.
------------
Retrieving Results for Index 6: hilton head island.
First hotel for the hilton head island is Hampton Inn Hilton Head.
------------
Retrieving Results for Index 7: uvira.
First hotel for the uvira is Auberge Eden City.
------------
Retrieving Results for Index 8: balaka.
First hotel for the balaka

In [24]:
hotel_df

,City,Country,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed(mph),Date,Hotel Nearby,Hotel Name
0,santa barbara,US,34.73,-120.03,73.00,53,1,4.70,1603490037,,Upper Ballard Campground
1,salalah,OM,17.02,54.09,75.20,44,0,2.24,1603489665,,HAMDAN PLAZA HOTEL
2,san jose,US,37.34,-121.89,75.00,43,20,4.70,1603489777,,The Westin San Jose
3,laguna,US,38.42,-121.42,79.00,15,1,4.70,1603489646,,Holiday Inn Express & Suites Elk Grove Central...
4,noumea,NC,-22.28,166.46,82.40,54,0,4.70,1603489510,,Hôtel Le Lagon
5,kalabo,ZM,-14.97,22.68,73.74,26,4,3.67,1603490151,,Sikakubete Guest House Kalabo Town Council
6,hilton head island,US,32.22,-80.75,78.80,69,20,4.70,1603489934,,Hampton Inn Hilton Head
7,uvira,CD,-3.41,29.15,74.52,60,6,3.74,1603490210,,Auberge Eden City
8,balaka,MW,-14.98,34.96,73.87,47,0,0.65,1603490225,,Sun Village Lodge


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='650px', margin='0 auto 0 auto', padding='1px', wi…